In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.trend import AroonIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
#from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator
import time
from IPython.display import clear_output
from datetime import datetime
from scipy import stats

start=(datetime.now())

sip500=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
sip400=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0].iloc[:,1])
sip600=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1].iloc[:,1])
payload=np.append(sip500,sip400)
payload=np.append(payload,sip600)
payload=np.unique(payload)
payload=payload[payload!='BRK.B']
payload=payload[payload!='JW.A']
payload=payload[payload!='BF.B']

#print(payload)


class algo:
    start="2010-01-04"
    end='2021-12-04'
    interval='1wk'

    dff=pd.DataFrame()
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        try:
            df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())
            df['stock']=self.stock
            #df['log_current']=np.log(df.Close/df.Open)
            df['log_current']=((df.Close-df.Open)/df.Open)
            #output = stock data from start to end
            clear_output(wait=False)
        except:
            pass
    
        return(df)
    
    
    def stockloop():
        l1=0
        l2=len(payload)
        dff=algo.dff
        boom=pd.DataFrame()
        for i in range (l1,l2):
            try:
                rawdata=algo(payload[i]).getdata()
                dff=pd.concat([dff,rawdata])
                print(i)
            except Exception as e:
                print(e)
                pass
        dff.set_index([dff.stock,dff.index],inplace=True)
        return(dff)

dff=algo.stockloop()
#dff.to_csv('raw.csv')
uniq=(dff.index.unique(level=0))

finish = datetime.now()
print('Duration: {}'.format(finish - start))
print(dff)
#time.sleep(6000)

1500
Duration: 0:12:38.964232
                       Open       High        Low      Close  Adj Close  \
stock Date                                                                
A     2010-01-04  22.453505  22.625179  21.745352  22.031473  20.134981   
      2010-01-11  22.088697  22.432047  21.494993  21.766809  19.893101   
      2010-01-18  21.716738  22.253220  20.808298  20.865522  19.069397   
      2010-01-25  21.044350  21.208870  19.971388  20.050072  18.324142   
      2010-02-01  20.143063  21.380545  20.121601  20.865522  19.069397   
...                     ...        ...        ...        ...        ...   
ZYXI  2021-11-01  12.550000  14.500000  12.110000  14.070000  14.070000   
      2021-11-08  14.130000  14.910000  13.910000  14.600000  14.600000   
      2021-11-15  14.630000  15.660000  14.410000  14.580000  14.580000   
      2021-11-22  14.520000  14.630000  12.610000  12.860000  12.860000   
      2021-11-29  12.980000  13.340000  11.680000  11.850000  11.85000

In [28]:
start=(datetime.now())
from ta.volume import ChaikinMoneyFlowIndicator
from ta import add_all_ta_features
from ta.utils import dropna
#print(dff)
class algo2:
    dfmass=pd.DataFrame()
    def __init__(self,stock):
        self.stock=stock

    def indicators(self):
        
        df=self.stock
        df['Open1']=df.Open.shift(1)
        df['Close1']=df.Close.shift(1)
        df['Low1']=df.Low.shift(1)
        df['High1']=df.High.shift(1)
        df['Volume1']=df.Volume.shift(1)
        df['Chng']=df.Close/df.Open-1
        df=df.dropna()
        #df['DR1']=df.Chng.shift(1)
        df['dlr']=np.log(df.Close/df.Open).shift(1)
        #df['DR2']=df.Chng.shift(1)+df.Chng.shift(2)
        #df['DR3']=df.DR2+df.Chng.shift(3)
        #df['V1']=df['Volume1']
        #df['V2']=df['V1']+df.Volume.shift(2)
        #df['V3']=df['V2']+df.Volume.shift(3)
        #df['DV']=df['Volume']*df.Chng
        #df['DV1']=df.DV.shift(1)
        #df['DV2']=df.DV1+df.DV.shift(2)
        #df['DV3']=df.DV2+df.DV.shift(3)
        df['DC']=df.Chng.shift(1)+df.Chng.shift(2)+df.Chng.shift(3)+df.Chng.shift(4)
        df['DW']=df.Volume.shift(1)+df.Volume.shift(2)+df.Volume.shift(3)+df.Volume.shift(4)
        df['DF']=df.DC*df.DW
        #df = add_all_ta_features(df, open="Open1", high="High1", low="Low1", close="Close1", volume="Volume1", fillna=True).dropna()
        
        df=df.dropna()
        return(df)
        
    def slicer(self):
        df=self.stock
        #ar_perc_t=np.array([])
        #ar_perc_b=np.array([])
        ar_perc_diff=np.array([])
        df_t=df[df['Chng']>0]
        df_b=df[df['Chng']<0]
        for c in range(14,len(df.columns)):
            #print(df.columns[c])
            t_ave=np.average(df_t.iloc[:,c])
            b_ave=np.average(df_b.iloc[:,c])
            #l_ave=np.append(l_ave,t_ave)
            perc_t=stats.percentileofscore(df.iloc[:,c],t_ave)
            perc_b=stats.percentileofscore(df.iloc[:,c],b_ave)
            perc_diff=perc_t-perc_b
            #ar_perc_t=np.append(ar_perc_t,perc_t)
            #ar_perc_b=np.append(ar_perc_b,perc_b)
            ar_perc_diff=np.append(ar_perc_diff,perc_diff)
        #print(ar_perc)
        return(ar_perc_diff)
        #output = stock data with indicators
       


    def stockloop():
        boom=np.array([])
        for i in uniq:
            try:
                rawdata=(dff.loc[i])
                ta=algo2(rawdata).indicators()
                #print(ta)
                percs=algo2(ta).slicer()
                #print(percs)
                boom=np.append(boom,percs)
                #print(boom)
            except:
                pass
        #boom=boom.reshape(len(uniq),len(ta.columns)-9)
        #boom=pd.DataFrame(boom)
        #boom.columns=ta.columns[9:len(ta.columns)]
        #boom.index=uniq
        return(boom)
df_out=algo2.stockloop()
print(df_out)
#df_out.to_csv('percentiles.csv')
#print(arf)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

[-6.64505673 -4.86223663 -0.16207455 ...  5.51053485 -0.81037277
 26.41815235]


In [29]:
print(len(df_out))
print(len(uniq))
dfw=df_out.reshape(1500,4)
dfw=pd.DataFrame(dfw)
dfw.to_csv('big_booba.csv')
print(dfw)
#dfw.columns=
for i in range (0,9):
    print(np.nanmean(dfw.iloc[:,i]))

6000
1500
              0         1          2          3
0     -6.645057 -4.862237  -0.162075 -12.479741
1      2.269044 -3.565640  -1.620746 -19.773096
2     16.326531  8.163265  40.816327   2.040816
3     -5.186386 -5.024311   2.106969  -4.051864
4     -9.886548 -3.565640  -0.324149  -5.024311
...         ...       ...        ...        ...
1495   0.162075 -0.162075   0.486224  -1.944895
1496  -4.700162 -5.348460  -2.269044  -5.510535
1497 -11.816193 -7.877462   0.000000  -7.658643
1498  -7.455429  2.917342   3.727715   2.269044
1499  -3.079417  5.510535  -0.810373  26.418152

[1500 rows x 4 columns]
-5.438731795803984
-2.8653950391554934
0.4183148741700253
-4.93230635964281


IndexError: single positional indexer is out-of-bounds